# Unidad 7: Aprendizaje por Refuerzo

En este *Notebook*, implementaremos un agente para jugar al <b>[FrozenLake](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).</b>

![alt text](https://gymnasium.farama.org/_images/frozen_lake.gif)


**Descripción**

El objetivo de este juego es ir <b>desde la posición inicial (S, Start) a la posición final (G, Goal)</b> caminando entre placas congeladas (F, Frozen) de un lago evitando los agujeros en el hielo (H, holes).

Los agujeros en el hielo se distribuyen en ubicaciones fijas al usar un mapa predeterminado. Existe la posibilidad de generar mapa con agujeros en ubicaciones aleatorias. Los mundos generados aleatoriamente siempre tendrán un camino hacia el objetivo.

Desde la posición inicial (S), el jugador realiza movimientos sobre las placas congeladas (F) hasta alcanzar el objetivo (G) o caer en un agujero (H). Pero el lago es resbaladizo (a menos que esta opción esté deshabilitada), por lo que a veces puede moverse perpendicularmente a la dirección deseada.

**Espacio de acciones:** las acciones están en el rango {0, 3}, indicando la dirección en la que se mueve el jugador.
*   0: Mover a la izquierda
*   1: Mover hacia abajo
*   2: Mover a la derecha
*   3: Mover hacia arriba

**Espacio de Observación:** la observación representa la posición actual del jugador como current_row * ncols + current_col (donde tanto la fila como la columna comienzan en 0). La observación se devuelve como un int().

Por ejemplo, la posición de la meta en el mapa 4x4 se puede calcular de la siguiente manera: 3 * 4 + 3 = 15. El número de observaciones posibles depende del tamaño del mapa.


**Estado Inicial:** el episodio comienza con el jugador en el estado [0] (ubicación [0, 0]).


**Recompensas:** se obtienen las siguientes recompensas:
*   Llegar a la meta: +1
*   Caer en hoyo: 0
*   Permanecer en hielo: 0


**Fin del Episodio:** el episodio termina si ocurre alguno de los eventos siguientes:

a) Terminación:
*   El jugador se mueve a un hoyo.
*   El jugador llega a la meta en max(nrow) * max(ncol) - 1 (ubicación [max(nrow)-1, max(ncol)-1]).

b) Truncamiento (al usar el contenedor time_limit):
*   La duración del episodio es de 100 minutos para el entorno 4x4 y de 200 minutos para el entorno FrozenLake8x8-v1.



## 1. Instalamos e importamos las dependencias en Google Colab y hacemos una prueba

Usaremos 3 librerías de Python:

1.   Random: para generar rúmeros aleatorios.
2.   Numpy: para nuestra Tabla-Q.
3.   Gym: para el entorno del FrozenLake. GYM Es una librería para experimentar con problemas y técnicas de RL creada por OpenAI.

A continuación inicializamos el entorno (sin resbalones) y vamos intentar dar 5 pasos sin caer en un agujero. A cada paso que damos imprimimos el estado del agente y del entorno.

In [1]:
import random

import numpy as np

try:
    import gymnasium as gym
except ImportError as err:
    !pip install gymnasium
    import gymnasium as gym


# Creamos el entorno para entrenar nuestro agente (el lago no es resbaladizo)
env = gym.make ("FrozenLake-v1", is_slippery=False, render_mode="ansi")

env.reset()     # Inicializamos el entorno
steps = 0  # Inicializamos los pasos que damos


while steps < 5:     # Vamos un dar un máximo de 5 pasos al azar
  action = env.action_space.sample()   # Generamos una acción al azar
  new_state, reward, terminated, truncated, info = env.step (action)
  print (env.render())                 # Imprimimos el estado del entorno
  steps += 1                      # Incrementamos los pasos
  print("\nEstado actual: ", new_state)

print("Ultima recompensa: ", reward)
print("Pasos: ", steps)

  (Up)
SFFF
FHFH
FFFH
HFFG


Estado actual:  0
  (Up)
SFFF
FHFH
FFFH
HFFG


Estado actual:  0
  (Down)
SFFF
FHFH
FFFH
HFFG


Estado actual:  4
  (Left)
SFFF
FHFH
FFFH
HFFG


Estado actual:  4
  (Down)
SFFF
FHFH
FFFH
HFFG


Estado actual:  8
Ultima recompensa:  0.0
Pasos:  5


## 2. Creamos la Tabla-Q y la inicializamos
- Ahora, debemos crear nuestra tabla Q, pero ¿cómo sabemos cuántas filas (estados) y columnas (acciones) necesitamos? Debemos obtener `action_size` y `state_size`.
- OpenAI Gym nos ayuda a obtener estos datos de un entorno (en nuestro caso, FrozenLake-v1): `env.action_space.n` y `env.observation_space.n`.

In [2]:
action_size = env.action_space.n
state_size = env.observation_space.n

print ("Action size: ", action_size)
print ("State size: ", state_size)

# Inicializamos nuestra Tabla-Q con state_size filas y action_size columnas
qtable = np.zeros ((state_size, action_size))
print (qtable)

Action size:  4
State size:  16
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Podemos ver que nuestra `qtable` tiene 4x4=16 filas con los estados/posiciones (0 estado inicial y 15 estado objetivo) y 4 columnas con las 4 posibles acciones (izquierda, derecha, arriba, abajo).

## 3. Inicializamos los hiperparámetros

A continuación inicializamos un conjunto de hiperparámetros que vamos a usar en el juego.

In [3]:
total_episodes = 20000       # Episodios totales
learning_rate = 0.7          # Factor de aprendizaje
max_steps = 99               # Número máximo de movimientos por episodio
gamma = 0.95                 # Ratio de descuento

# Parámetros de exploración
epsilon = 1.0                 # Ratio de exploración
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Ratio de caída exponencial durante la exploración

## 4. El algoritmo Q-Learning


* Q-learning es una técnica de aprendizaje por refuerzo que tiene como objetivo aprender una estrategia que le diga a un agente qué acción tomar bajo qué circunstancias.


* Para ello utiliza la ***Q-Function*** definida como:


<span style="font-size:20px">
$$\widehat{Q}(s, a) = Q(s, a) + \alpha \cdot \left [ R(s,a) + \left (\gamma \cdot \underset{{a}'}{max} Q({s}',{a}') \right ) - Q(s,a) \right ]$$
</span>


* Siendo:
    + $\widehat{Q}(s, a)$:El nuevo valor de $Q(s, a)$
    + $Q(s, a)$: El antiguo valor de $Q(s, a)$ o valor actual.
    + $\alpha$: Factor de aprendizaje (Learning Rate) que indica "*cuanto*" queremos aprender en cada acción.
    + $R(s,a)$: Recompensa por tomar la acción '$a$' desde el estado actual $s$.
    + $\gamma$: Factor de descuento.
    + $\underset{{a}'}{max} Q({s}',{a}')$: Valor de $Q(s, a)$ tras tomar la mejor acción posible.
    
    
* Este algoritmo puede aprender la estrategia optimizando la recompensa a corto o largo plazo o puede maximizar la recompensa a largo plazo aunque esto le suponga no realizar el movimiento más prometedor dado un estado. Para elegir entre ambas estrategias usamos el hiperparámetro $\gamma$ (factor de descuento) que tendrá que tener un valor cercano a $0$ si queremos tomar una estrategia a corto plazo y un valor cercano a $1$ si decidimos tomar una estrategia a largo plazo.


* En este algoritmo debemos tener en cuenta los siguientes puntos:
    1. El agente se ejecutara un número determinado de veces (episodios).
    2. Cada vez que se ejecute, el agente partirá del estado inicial $S$ por lo que hay que inicializar el entorno.
    3. Para cada ejecución, el agente calculará los valores de las acciones que puede tomar en cada uno de los estados con la Q-Function y actualizará el valor $Q(s,a)$ en la Tabla-Q.
    4. El agente en cada estado puede seleccionar las acciones de dos formas:
        + Explorando: Selecciona una acción al azar.
        + Explotando: Selecciona la mejor acción entre todas las posibles.
    5. El algoritmo se ejecuta hasta que el agente llega al estado final.
    
    
* Para determinar si el agente tiene que explorar o explotar en un determinado estado se define un parámetro conocido como "greedy_control" ($\in$) que no es más que la definición de una probabilidad de explotación o exploración; por ejemplo, si definimos $\in=0.1$, un 10% de las veces exploraremos y un 90% de las veces explotaremos el conocimiento adquirido por el sistema.

In [ ]:
# Lista de recompensas
rewards = []

# Reiniciamos la qtable (por si lo ejecutamos varias veces)
qtable = np.zeros ((state_size, action_size))

# Ciclo de ejecución sobre total_episodes
for episode in range(total_episodes):
    # Reiniciamos el entorno
    state, info = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range (max_steps):
        ## Elegimos un valor aleatorio para decidir si exploraramos o explotamos
        eps_greedy = random.uniform (0, 1)

        ## Si el valor aleatorio es mayor que epsilon --> explotación (tomamos la acción con el mayor valor de Q para este estado)
        if eps_greedy > epsilon:
            action = np.argmax (qtable[state,:])    # Elegimos una acción (action) en el estado actual (state)

        # En el otro caso --> exploración (elegimos cualquier acción al azar)
        else:
            action = env.action_space.sample()      # Elegimos una acción (action) en el estado actual (state)


        # Usamos la acción (action) y observamos: el nuevo estado (new_state) y la recompensa (reward)
        # 'env.step' nos devuelve 5 valores: obs, reward, terminated, truncated, info
        new_state, reward, terminated, truncated, info = env.step (action)
        done = terminated or truncated    # combinamos terminated y truncated en done


        # Actualizamos Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : indica todas las acciones que podemos tomar en el nuevo estado
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])

        total_rewards += reward

        # Nuestro nuevo estado es state
        state = new_state

        # Si e ha caído en un agujero o alcanzado el objetivo se termina el episodio
        if done == True:
            break

    # Reducimos epsilon (porque cada vez exploraremos menos)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp (-decay_rate*episode)
    rewards.append (total_rewards)


print ("Recompensa media global: " +  str(sum(rewards)/total_episodes), end='\n\n')
print ("La tabla-Q obtenida es:")
print (qtable)

Recompensa media global: 0.0

La tabla-Q obtenida es:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## 5. Usamos nuestra Tabla-Q para jugar a FrozenLake !
- Tras un largo periodo de aprendizade nuestra Tabla-Q ya puede usarse para jugar a FrozenLake". Vamos a jugar 5 veces.


In [ ]:
env.reset()

for episode in range(5):      # Juega en 5 intentos
    state, info = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODIO: ", episode)

    for step in range (max_steps):

        # Toma la acción que tiene la mayor recompensa futura estimada en ese estado
        action = np.argmax (qtable[state,:])

        new_state, reward, terminated, truncated, info = env.step (action)
        done = terminated or truncated # combinamos terminated o truncated en done

        if done:
            if new_state == 15:
                print ("Alcanzamos el objetivo 🏆")
            else:
                print ("Caímos en un agujero ☠️")

            print ("Número de pasos: ", step)

            break
        state = new_state
env.close()

****************************************************
EPISODIO:  0
****************************************************
EPISODIO:  1
****************************************************
EPISODIO:  2
****************************************************
EPISODIO:  3
****************************************************
EPISODIO:  4


## 6. Vuelve a la sección (3) y modifica los hiperparámetros para ver qué ocurre.
- Modifica: total_episodes, learning_rate, gamma, etc.
- Ejecuta el paso 4 nuevamente para ver qué se obtiene.
- **Haz un breve estudio de sensibilidad de los parámetros, en este escenario, indicando cuáles son las mejores elecciones y cuál influye más en los resultados !**

In [ ]:
# Incluye aquí el código o las gráficas que muestren el estudio de sensibilidad de parámetros.



## 7. Genera un nuevo código usando ahora el algoritmo SARSA !

* El algoritmo SARSA (State–Action–Reward–State–Action) es una técnica de aprendizaje por refuerzo similar al Q-learning, con la diferencia de que en la función Q no selecciona el valor máximo esperado $\underset{{a}'}{max} Q({s}',{a}')$, si no que selecciona la acción que hubiesemos tomado en ese nuevo estado ${s}'$ con la política actual que estamos utilizando.


* Veamos como queda la función de Estado-Acción:


<span style="font-size:20px">
$$\widehat{Q}(s, a) = Q(s, a) + \alpha \cdot \left [ R(s,a) + \left (\gamma \cdot Q({s}',{a}') \right ) - Q(s,a) \right ]$$
</span>


* Siendo:
    + $\widehat{Q}(s, a)$:El nuevo valor de $Q(s, a)$
    + $Q(s, a)$: El antiguo valor de $Q(s, a)$ o valor actual.
    + $\alpha$: Factor de aprendizaje (Learning Rate) que indica "*cuanto*" queremos aprender en cada acción.
    + $R(s,a)$: Recompensa por tomar la acción '$a$' desde el estado actual $s$.
    + $\gamma$: Factor de descuento.
    + $Q({s}',{a}')$: Valor de la mejor acción que hubiesemos tomado en el nuevo estado ${a}'$


* Como se puede observar en el Pseudocódigo la diferencia fundamental frente al Q-Learning es que se selecciona la mejor acción a tomar en el estado actual (igual que en Q-Learning) pero en la función se toma como valor $Q({s}',{a}')$, es decir, la acción que se tomaría en el estado al que nos moveríamos a continuación.




In [ ]:
# Inclumos los hiperparámetros a continuación en el código para simplificar las modificaciones
total_episodes = 20000       # Episodios totales
learning_rate = 0.7          # Factor de aprendizaje
max_steps = 99               # Número máximo de movimientos por episodio
gamma = 0.95                 # Ratio de descuento

# Parámetros de exploración
epsilon = 1.0                 # Ratio de exploración
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Ratio de caída exponencial durante la exploración


# Incluye tu nuevo código de SARSA a continuación...


## 8. Haz un nuevo estudio de la sensibilidad de los parámetros sobre SARSA


In [ ]:
# Incluye aquí el código o las gráficas que muestren el estudio de sensibilidad de parámetros.

## 9. Compara ambos algoritmos (QL vs. SARSA) en un entorno resbaladizo

In [ ]:
# Incluye aquí los resultados (código o gráficas) de la comparación...